In [2]:
import os
import csv
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [14]:
# helper functions

def series_append(series, list, keys):
    for i in range(64):
        series[keys[i]].append(float(list[i]))
    return series

def load_series(filename):
    with open(filename, 'r') as csv_in:
        csv_file = list(csv.reader(csv_in))
        series = {}
        keys = csv_file[0]
        for key in keys: series[key] = []
        for i in range(2, len(csv_file), 2):
            series = series_append(series, csv_file[i], keys)
        return [series, int((len(csv_file) - 2) / 2)]


def unroll(series):
    l = []
    for key in series:
        if (key[-1] == 'v' or key[-1] == 'z'): continue
        l += (series[key])
    return l

def E3(pred, truth):
    num_correct = 0
    for i in range(len(pred)):
        if (abs(pred[i] - truth[i]) <= 3.5):
            num_correct += 1
    return (100.0 * float(num_correct) / float(len(pred)))

def E2(pred, truth):
    num_correct = 0
    for i in range(len(pred)):
        if (abs(pred[i] - truth[i]) <= 2.5):
            num_correct += 1
    return (100.0 * float(num_correct) / float(len(pred)))

def E1(pred, truth):
    num_correct = 0
    for i in range(len(pred)):
        if (abs(pred[i] - truth[i]) <= 1.5):
            num_correct += 1
    return (100.0 * float(num_correct) / float(len(pred)))

def E0(pred, truth):
    num_correct = 0
    for i in range(len(pred)):
        print('PREDICTION: ' + str(pred[i]))
        print('ACTUAL: ' + str(truth[i]))
        if (abs(pred[i] - truth[i]) <= 0.5):
            num_correct += 1
    return (100.0 * float(num_correct) / float(len(pred)))

In [8]:
# loading data

raw_X_train_list = []
smoothed_X_train_list = []
trans_X_train_list = []
final_X_train_list = []

y_train_list = []

raw_X_test_list = []
smoothed_X_test_list = []
trans_X_test_list = []
final_X_test_list = []

y_test_list = []

with open("..\\test_examples.txt") as t:
    test_examples = t.readlines()

for example in test_examples:
    raw_series, num_frames = load_series("..\\time_series\\Time_normalized_stages\\1_unprocessed\\" + example[:-1])
    smoothed_series, num_frames = load_series("..\\time_series\\Time_normalized_stages\\2_smoothed\\" + example[:-1])
    trans_series, num_frames = load_series("..\\time_series\\Time_normalized_stages\\3_translation\\" + example[:-1])
    final_series, num_frames = load_series("..\\time_series\\Time_normalized_stages\\4_final\\" + example[:-1])

    raw_X_test_list.append(unroll(raw_series))
    smoothed_X_test_list.append(unroll(smoothed_series))
    trans_X_test_list.append(unroll(trans_series))
    final_X_test_list.append(unroll(final_series))
    
    y_test_list.append([float(example[4]) / 9, float(example[7]) / 9, float(example[10]) / 9, float(example[13]) / 9, float(example[16]) / 9])

raw_X_test = torch.tensor(raw_X_test_list)
smoothed_X_test = torch.tensor(smoothed_X_test_list)
trans_X_test = torch.tensor(trans_X_test_list)
final_X_test = torch.tensor(final_X_test_list)


y_test = torch.tensor(y_test_list)

print(raw_X_test.size())
print(smoothed_X_test.size())
print(trans_X_test.size())
print(final_X_test.size())
print(y_test.size())
print('\n\n')

with open("..\\training_examples.txt") as t:
    training_examples = t.readlines()

for example in training_examples:
    raw_series, num_frames = load_series("..\\time_series\\Time_normalized_stages\\1_unprocessed\\" + example[:-1])
    smoothed_series, num_frames = load_series("..\\time_series\\Time_normalized_stages\\2_smoothed\\" + example[:-1])
    trans_series, num_frames = load_series("..\\time_series\\Time_normalized_stages\\3_translation\\" + example[:-1])
    final_series, num_frames = load_series("..\\time_series\\Time_normalized_stages\\4_final\\" + example[:-1])

    raw_X_train_list.append(unroll(raw_series))
    smoothed_X_train_list.append(unroll(smoothed_series))
    trans_X_train_list.append(unroll(trans_series))
    final_X_train_list.append(unroll(final_series))
    
    y_train_list.append([float(example[4]) / 9, float(example[7]) / 9, float(example[10]) / 9, float(example[13]) / 9, float(example[16]) / 9])

raw_X_train = torch.tensor(raw_X_train_list)
smoothed_X_train = torch.tensor(smoothed_X_train_list)
trans_X_train = torch.tensor(trans_X_train_list)
final_X_train = torch.tensor(final_X_train_list)

y_train = torch.tensor(y_train_list)

print(raw_X_train.size())
print(smoothed_X_train.size())
print(trans_X_train.size())
print(final_X_train.size())
print(y_train.size())

torch.Size([93, 224])
torch.Size([93, 224])
torch.Size([93, 224])
torch.Size([93, 224])
torch.Size([93, 5])



torch.Size([371, 224])
torch.Size([371, 224])
torch.Size([371, 224])
torch.Size([371, 224])
torch.Size([371, 5])


In [9]:
# setting up FCNN

def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_normal_(m.weight)
        m.bias.data.fill_(0.01)

input_size = 224
hidden_layer_size_1 = 450
hidden_layer_size_2 = 400
hidden_layer_size_3 = 330
hidden_layer_size_4 = 250
hidden_layer_size_5 = 150
hidden_layer_size_6 = 50
output_size = 5
batch_size = 371
learning_rate = 0.001

model = nn.Sequential(nn.Linear(input_size, hidden_layer_size_1),
                        nn.ReLU(),
                        nn.Linear(hidden_layer_size_1, hidden_layer_size_2),
                        nn.ReLU(),
                        nn.Linear(hidden_layer_size_2, hidden_layer_size_3),
                        nn.ReLU(),
                        nn.Linear(hidden_layer_size_3, hidden_layer_size_4),
                        nn.ReLU(),
                        nn.Linear(hidden_layer_size_4, hidden_layer_size_5),
                        nn.ReLU(),
                        nn.Linear(hidden_layer_size_5, hidden_layer_size_6),
                        nn.ReLU(),
                        nn.Linear(hidden_layer_size_6, output_size),
                        nn.Sigmoid())

loss_function = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
# training and evaluating raw series

num_epochs = 250

min_loss = 1.0

training_cycles = 1

raw_E0 = []
raw_E1 = []
raw_E2 = []
raw_E3 = []

for i in range(training_cycles):
    model.apply(init_weights)
    test_losses = []
    for epoch in range(num_epochs):
        pred_y = model(raw_X_train)
        loss = loss_function(pred_y, y_train)

        test_y = model(raw_X_test)
        test_loss = loss_function(test_y, y_test)
        test_losses.append(test_loss.item())

        model.zero_grad()
        loss.backward()

        optimizer.step()

    if (test_losses [-1] < min_loss):
        min_loss = test_losses[-1]
        test_y = model(raw_X_test)

        raw_E0.clear()
        raw_E1.clear()
        raw_E2.clear()
        raw_E3.clear()

        print(model(raw_X_test))
        
        raw_E0 = E0(test_y, y_test.tolist())
        raw_E1 = E1(test_y, y_test.tolist())
        raw_E2 = E2(test_y, y_test.tolist())
        raw_E3 = E3(test_y, y_test.tolist())
        

# TODO FIX ERROR METRICS IMPLEMENTATION

print('MSE: ' + str(min_loss))
print('RAW E0: ' + str(raw_E0))
print('RAW E1: ' + str(raw_E1))
print('RAW E2: ' + str(raw_E2))
print('RAW E3: ' + str(raw_E3))